# Setup Code

In [30]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import json
from logging import exception
from twelvedata import TDClient
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
import time

# Load API keys
with open('work\key.json', 'r') as file:
    config = json.load(file)

pw = config['email_password']  # email password

keys = config['keys']  # List of API keys

# Initialize clients
clients = [TDClient(apikey=key) for key in keys]

with open("work/tickers.txt") as f:
    lines = f.readlines()

tickers = [line.strip() for line in lines]

# Get Data

In [14]:
# Calculate moving averages for 5, 10, and 20 days
def moving_average(prices, days):
    if len(prices) >= days:
        return sum(prices[:days]) / days
    return None  # Not enough data for this period


def get_data(stocks, interval, client):
    ts = client
    ts = client.time_series(
        symbol=stocks,
        interval=interval,
        outputsize=30,
        timezone="America/New_York",
    )

    data = ts.with_macd().with_heikinashicandles().as_json()
    values = {ticker: [] for ticker in stocks.split(',')}
    closing_prices = {ticker: [] for ticker in stocks.split(',')}
    
    for ticker, entries in data.items():
        for entry in entries:
            closing_prices[ticker].append(float(entry["heikincloses"]))

    for ticker, entries in data.items():

        # Traverse the entries from most recent (index 0) to least recent (index -1)
        for idx in range(len(entries)):
            entry = entries[idx]
            datetime_str = entry["datetime"]
            macd = float(entry["macd"])
            macd_signal = float(entry["macd_signal"])
            macd_hist = float(entry["macd_hist"])
            open_price = float(entry["heikinopens"])
            close_price = float(entry["heikincloses"])
            high = float(entry["heikinhighs"])
            low = float(entry["heikinlows"])
            
            ma_5 = sum(closing_prices[ticker][idx:idx+5]) / (5) if idx + 5 <= len(closing_prices[ticker]) else None
            ma_10 = sum(closing_prices[ticker][idx:idx+10]) / (10) if idx + 10 <= len(closing_prices[ticker]) else None
            ma_20 = sum(closing_prices[ticker][idx:idx+20]) / (20) if idx + 20 <= len(closing_prices[ticker]) else None

            values[ticker].append({
                "datetime": datetime_str,
                "macd": macd,
                "macd_signal": macd_signal,
                "macd_hist": macd_hist,
                "open": open_price,
                "close": close_price,
                "high": high,
                "low": low,
                "ma_5": ma_5,
                "ma_10": ma_10,
                "ma_20": ma_20,
            })

    return values

# Evalute Stocks

In [26]:
def evaluate_stock_criteria(data, ticker):
    result = {"ticker": ticker, "cross": "H", "trend": "H", "predicted_cross": "H", "moving_average": "H"}

    # Detect MACD crossover in the last 2 days
    crosses = []
    for i in range(len(data) - 1):
        current = data[i]
        next_data = data[i + 1]

        current_macd = float(current["macd"])
        current_macd_signal = float(current["macd_signal"])
        next_macd = float(next_data["macd"])
        next_macd_signal = float(next_data["macd_signal"])

        if (current_macd < current_macd_signal and next_macd > next_macd_signal):
            crosses.append({"datetime": next_data["datetime"], "type": "death cross"})
        elif (current_macd > current_macd_signal and next_macd < next_macd_signal):
            crosses.append({"datetime": next_data["datetime"], "type": "golden cross"})

    # Check for recent cross within the last 2 days
    now = datetime.now()
    yesterday = now - timedelta(days=3)
    recent_cross = None
    # print("Crosses: ", crosses)

    for cross in crosses:
        cross_datetime = datetime.strptime(cross["datetime"], "%Y-%m-%d")
        if cross_datetime >= yesterday:
            recent_cross = cross
            break

    # Update result based on recent cross
    if recent_cross:
        if recent_cross["type"] == "golden cross":
            result["cross"] = "B"
        elif recent_cross["type"] == "death cross":
            result["cross"] = "S"

    # Predict tomorrow's trend
    last_two = data[:2][::-1]
    x = np.array([i for i in range(len(last_two))]).reshape(-1, 1)
    y_macd = np.array([float(point["macd"]) for point in last_two])
    y_signal = np.array([float(point["macd_signal"]) for point in last_two])

    model_macd = LinearRegression().fit(x, y_macd)
    model_signal = LinearRegression().fit(x, y_signal)

    next_x = np.array([[len(last_two)]])
    predicted_macd = model_macd.predict(next_x)[0]
    predicted_signal = model_signal.predict(next_x)[0]

    result["trend"] = "B" if predicted_macd > predicted_signal else "S"

    # Determine if a crossover is predicted for tomorrow
    last_macd = float(last_two[-1]["macd"])
    last_signal = float(last_two[-1]["macd_signal"])

    if (last_macd < last_signal and predicted_macd > predicted_signal):
        result["predicted_cross"] = "B"
    elif (last_macd > last_signal and predicted_macd < predicted_signal):
        result["predicted_cross"] = "S"
    else:
        result["predicted_cross"] = "H"

    # Evaluate price movement (open, high, low, close)
    most_recent = data[0]
    second_recent = data[1]
    prev_open = float(data[1]["open"])
    prev_close = float(data[1]["close"])
    close_price = float(most_recent["close"])
    high_price = float(most_recent["high"])
    low_price = float(most_recent["low"])
    open_price = float(most_recent["open"])

    real_low = min(open_price,low_price,high_price,close_price)
    real_high = max(open_price,low_price,high_price,close_price)
    
    green_up_wick = (real_high - close_price)
    green_bottom_wick = (open_price - real_low)
    
    red_up_wick = (real_high - open_price)
    red_bottom_wick = (close_price - real_low)
    
    if green_up_wick > green_bottom_wick * 1.5:
        green_double_wick = False
    else:
        green_double_wick = True

    if red_up_wick > red_bottom_wick * 1.5:
        red_double_wick = False
    else:
        red_double_wick = True
    

    if close_price > open_price and real_low >= open_price * 0.9975 and not green_double_wick:
        result["sym"] = "B"
    elif close_price < open_price and real_high >= open_price * 1.0025 and not red_double_wick:
        result["sym"] = "S"
    else:
        result["sym"] = "H"

    ma5 = float(data[0]["ma_5"])
    ma10 = float(data[0]["ma_10"])
    ma20 = float(data[0]["ma_20"])
    
    if ma5 and ma10 and ma20:
        if ma5 > ma10 and ma10 > ma20:
            result["moving_average"] = "B"  # 5-day > 10-day > 20-day
        elif ma5 < ma10 and ma10 < ma20:
            result["moving_average"] = "S"  # 5-day < 10-day < 20-day
        else:
            result["moving_average"] = "H"  # No specific trend
    
    return result

In [31]:
# Important lists
important_buy_list = []
important_sell_list = []

key_index = 0  # Start with the first client
batch_size = 4

for i in range(0, len(tickers), batch_size):
    batch_tickers = tickers[i:i + batch_size]
    client = clients[key_index]
    values = get_data(",".join(batch_tickers), "1day", client)
    
    if values:
        for ticker in batch_tickers:
            evaluation = evaluate_stock_criteria(values[ticker], ticker)
            # Count the number of "B" and "S" evaluations
            b_count = sum(1 for key in ["cross", "sym", "moving_average"] if evaluation[key] == "B")
            s_count = sum(1 for key in ["cross", "sym", "moving_average"] if evaluation[key] == "S")
            
            stock_info = f"{evaluation['ticker']}: {evaluation['cross']}, {evaluation['sym']}, {evaluation['moving_average']}"
            
            print(evaluation)
            if b_count > 2:
                important_buy_list.append(stock_info)
            elif s_count > 2:
                important_sell_list.append(stock_info)

    key_index += 1
    
    # If all clients have been used, reset the index and sleep for 61 seconds
    if key_index >= len(clients):
        key_index = 0
        print("Processed one full cycle of keys. Sleeping for 61 seconds.")
        time.sleep(61)

{'ticker': 'MSFT', 'cross': 'B', 'trend': 'B', 'predicted_cross': 'H', 'moving_average': 'S', 'sym': 'B'}
{'ticker': 'AAPL', 'cross': 'H', 'trend': 'S', 'predicted_cross': 'H', 'moving_average': 'S', 'sym': 'S'}
{'ticker': 'NVDA', 'cross': 'H', 'trend': 'S', 'predicted_cross': 'H', 'moving_average': 'S', 'sym': 'B'}
{'ticker': 'AMZN', 'cross': 'H', 'trend': 'B', 'predicted_cross': 'H', 'moving_average': 'H', 'sym': 'B'}


In [45]:
print("Ticker: MACD Cross / SYM / SMA")
print("\nBuy:")
for stock in important_buy_list:
    ticker = stock.split(":")[0]
    cross, trend, ma = stock.split(":")[1].split(", ")
    print(f"{ticker}: {cross} {trend} {ma}")

print("\nSell:")
for stock in important_sell_list:
    ticker = stock.split(":")[0]
    cross, trend, ma = stock.split(":")[1].split(", ")
    print(f"{ticker}: {cross} {trend} {ma}")


def send_email(recipient, subject, body):
    port = 465
    smtp_server = "smtp.gmail.com"
    sender_email = "calvinz168@gmail.com"
    password = pw

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient

    msg['Subject'] = subject
    body = MIMEText(body)
    msg.attach(body)

    server = smtplib.SMTP_SSL(smtp_server, port)

    server.login(sender_email, password)
    server.sendmail(sender_email, recipient, msg.as_string())
    server.quit()
    
# Convert the lists to strings for email content
buy_stocks_string = "\n".join(important_buy_list)
sell_stocks_string = "\n".join(important_sell_list)

# Prepare email content
email_body = f"Ticker: MACD Cross / SYM / SMA\nBuy:\n{buy_stocks_string}\n\nSell:\n{sell_stocks_string}"

# Send email with the combined summary
send_email("calvinz168@gmail.com", f"Stocks Summary - {datetime.now().date()}", email_body)
send_email("davisz@gmx.com", f"Stocks Summary - {datetime.now().date()}", email_body)

Ticker: MACD Cross / SYM / SMA

Buy:

Sell:
